# HUFFMAN

In [1]:
import os
import heapq
from bitarray import bitarray #need to install bitarray

### Opening file and creating frequency dictionary 

In [2]:
with open("norm_wiki_sample.txt",'r+') as file:
    text = file.read()
    text = text.rstrip()
frequency_dict = dict()

for char in text:
    if char in frequency_dict:
        frequency_dict[char] += 1
    else:
        frequency_dict[char] = 1

frequency_dict_sorted = dict(sorted(frequency_dict.items(), key=lambda item: item[1]))

### Node class with comparison method

In [3]:
class HeapNode:
    def __init__(self, char, freq):
        self.char = char
        self.freq = freq
        self.left = None
        self.right = None
    
    def __lt__(self, other):
        return self.freq < other.freq

    def __eq__(self, other):
        if(other == None):
            return False
        if(not isinstance(other, HeapNode)):
            return False
        return self.freq == other.freq

### Recusive tree creator function

In [4]:
def code_creator(root, current_code):
    if(root == None):
        return

    if(root.char != None):
        codes[root.char] = current_code
        reverse_mapping[current_code] = root.char
        return

    code_creator(root.left, current_code + "0")
    code_creator(root.right, current_code + "1")

### Creating huffman code

In [5]:
heap = []
codes = {}
reverse_mapping = {}

for key in frequency_dict_sorted:
    node = HeapNode(key, frequency_dict_sorted[key])
    heapq.heappush(heap, node)

while(len(heap)>1):
    LEFTnode = heapq.heappop(heap)
    RIGHTnode = heapq.heappop(heap)

    merged = HeapNode(None, LEFTnode.freq + RIGHTnode.freq)
    merged.left = LEFTnode
    merged.right = RIGHTnode

    heapq.heappush(heap, merged)

root = heapq.heappop(heap)
current_code = ""    
code_creator(root, current_code)
print(codes)

{'e': '000', 'm': '00100', 'y': '001010', 'k': '0010110', '4': '001011100', 'x': '001011101', '5': '001011110', '3': '001011111', 's': '0011', 'w': '010000', 'b': '010001', 'c': '01001', 'r': '0101', 'o': '0110', 'n': '0111', 'i': '1000', 'd': '10010', '2': '10011000', '9': '10011001', 'v': '1001101', 'g': '100111', 't': '1010', 'p': '101100', 'f': '101101', 'l': '10111', 'a': '1100', 'h': '11010', '8': '110110000', 'j': '110110001', '0': '11011001', 'q': '1101101000', 'z': '1101101001', '6': '1101101010', '7': '1101101011', '1': '11011011', 'u': '110111', ' ': '111'}


In [6]:
encoded_text = bitarray()
for char in text:
    encoded_text.extend(codes[char])
with open("norm_wiki_sample_huffman_encoded",'wb') as encodedfile:
    encoded_text.tofile(encodedfile)

In [7]:
print("Number of bits of compressed text:",len(encoded_text))
print("Number of bits of shortest possible fixed-length encoding (6):",len(text)*6)

Number of bits of compressed text: 46489711
Number of bits of shortest possible fixed-length encoding (6): 64733640


In [8]:
print("Compression ratio when we assuming standard 8 bits:",(len(text)*8)/len(encoded_text))
print("Compression ratio when we assuming shortest possible fixed-length encoding (6):",(len(text)*6)/len(encoded_text))

Compression ratio when we assuming standard 8 bits: 1.8565725220361124
Compression ratio when we assuming shortest possible fixed-length encoding (6): 1.3924293915270844


In [9]:
decryptionary = dict()
for k,v in codes.items():
    decryptionary[v]=k

In [10]:
with open("norm_wiki_sample_huffman_encoded",'rb') as file_to_decode:
    text_to_decode = bitarray()
    text_to_decode.fromfile(file_to_decode)

decrypted_text = ""
b_char = ""

for bit in text_to_decode:
    
    if bit:
        binary = "1"
    else:
        binary = "0"
        
    b_char += binary
    if(b_char in decryptionary):
        decrypted_text+=decryptionary[b_char]
        b_char = ""

In [11]:
with open("norm_wiki_sample_huffman_decoded.txt",'w') as file:
    file.write(decrypted_text)

### Comparison of source with decoded text

In [12]:
if(text == decrypted_text):
    print("Successful Decryption")

Successful Decryption


# LZW

In [13]:
def LZW_compression(dictionary, data,maximum_table_size):
    string = ""
    dictionary_size = len(dictionary) 
    compressed_data = []
    for symbol in data:                     
        string_plus_symbol = string + symbol
        if string_plus_symbol in dictionary: 
            string = string_plus_symbol
        else:
            compressed_data.append(dictionary[string])
            if(len(dictionary) <= maximum_table_size):
                dictionary[string_plus_symbol] = dictionary_size
                dictionary_size += 1
            string = symbol
    
    if string in dictionary:
        compressed_data.append(dictionary[string])
    
    return compressed_data

In [14]:
def LZW_decompression(compressed_data):
    decompressed_data = ""
    next_code = 256
    string = ""
    for code in compressed_data:
        if not (code in dictionary):
            dictionary[code] = string + (string[0])
        decompressed_data += dictionary[code]
        if not(len(string) == 0):
            dictionary[next_code] = string + (dictionary[code][0])
            next_code += 1
        string = dictionary[code]
        
    return decompressed_data

# LZW for norm_wiki.txt

In [15]:
from struct import *

n = 15 
maximum_table_size = pow(2,int(n)) 

with open("wiki_sample.txt") as file:                 
    data = file.read()                      

dictionary_size = 256                  
dictionary = {chr(i): i for i in range(dictionary_size)}    

compressed_data = LZW_compression(dictionary, data, maximum_table_size)

with open("wiki_sample" + ".lzw", "wb") as output_file:
    for data in compressed_data:
        output_file.write(pack('>H',int(data))) 
        
compressed_data_dec = []
with open("wiki_sample.lzw","rb") as file:
    while True:
        rec = file.read(2)
        if len(rec) != 2:
            break
        (data, ) = unpack('>H', rec)
        compressed_data_dec.append(data)

dictionary_size = 256
dictionary = dict([(x, chr(x)) for x in range(dictionary_size)])

decompressed_data = LZW_decompression(compressed_data_dec)

with open("wiki_sample" + "_dec.txt", "w") as output_file:
    for data in decompressed_data:
        output_file.write(data)   

In [16]:
print(compressed_data_dec[:100])

[64, 64, 49, 53, 49, 52, 32, 65, 108, 98, 101, 114, 116, 32, 111, 102, 32, 80, 114, 117, 115, 115, 105, 97, 32, 40, 32, 49, 55, 32, 77, 97, 121, 282, 52, 57, 48, 32, 50, 292, 286, 114, 99, 104, 282, 53, 54, 56, 32, 41, 32, 119, 97, 115, 32, 116, 104, 101, 32, 108, 308, 268, 71, 114, 97, 110, 100, 285, 316, 266, 269, 271, 311, 313, 84, 101, 117, 116, 111, 110, 105, 99, 32, 75, 335, 103, 104, 116, 309, 44, 306, 104, 111, 32, 97, 102, 116, 325, 99, 334]


In [17]:
out = ""
for i in range(100):
    if compressed_data_dec[i] < 256:
        out += chr(compressed_data_dec[i])
    else:
        out += str(compressed_data_dec[i])
out

'@@1514 Albert of Prussia ( 17 May282490 2292286rch282568 ) was the l308268Grand285316266269271311313Teutonic K335ght309,306ho aft325c334'

In [18]:
decompressed_data[:100]

'@@1514 Albert of Prussia ( 17 May 1490 20 March 1568 ) was the last Grand Master of the Teutonic Kni'

In [19]:
with open("wiki_sample.txt") as source, open("wiki_sample_dec.txt") as decoded:
    text1 = source.read()
    text2 = decoded.read()
    
if(text1 == text2):
    print("Successful Decryption")

Successful Decryption


# LZW for norm_wiki_sample.txt

In [20]:
with open("norm_wiki_sample.txt") as file:                 
    data = file.read()                      

dictionary_size = 256                  
dictionary = {chr(i): i for i in range(dictionary_size)}    

compressed_data = LZW_compression(dictionary, data, maximum_table_size)

with open("norm_wiki_sample" + ".lzw", "wb") as output_file:
    for data in compressed_data:
        output_file.write(pack('>H',int(data))) 
        
compressed_data_dec = []
with open("norm_wiki_sample.lzw","rb") as file:
    while True:
        rec = file.read(2)
        if len(rec) != 2:
            break
        (data, ) = unpack('>H', rec)
        compressed_data_dec.append(data)

dictionary_size = 256
dictionary = dict([(x, chr(x)) for x in range(dictionary_size)])

decompressed_data = LZW_decompression(compressed_data_dec)

with open("norm_wiki_sample" + "_dec.txt", "w") as output_file:
    for data in decompressed_data:
        output_file.write(data)   

In [21]:
out = ""
for i in range(100):
    if compressed_data_dec[i] < 256:
        out += chr(compressed_data_dec[i])
    else:
        out += str(compressed_data_dec[i])
out

' albert of prussia 17 may274490 2284278rch274568 was the l298262grand277306260263265301303teutonic k325ght299who256f320r c324v260ting300337l322302'

In [22]:
decompressed_data[:100]

' albert of prussia 17 may 1490 20 march 1568 was the last grand master of the teutonic knights who a'

In [23]:
with open("norm_wiki_sample.txt") as source, open("norm_wiki_sample_dec.txt") as decoded:
    text1 = source.read()
    text2 = decoded.read()
    
if(text1 == text2):
    print("Successful Decryption")

Successful Decryption
